In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
main_url = 'https://www.kathimerini.gr/'
start_url = 'https://www.kathimerini.gr/politics/page/'
start_page = 2
end_page = 27

In [ ]:
response = requests.get(start_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [ ]:
teasers_list = []
for page_num in range(start_page, end_page + 1):
    page_url = start_url + str(page_num)
    response = requests.get(page_url)
    doc = BeautifulSoup(response.text, 'html.parser')

    teaser_articles_list = doc.find_all('div', {'class': 'nx-article'})

    for teaser in teaser_articles_list:
        story_dict = {}
        try:
            link = teaser.find('a')['href']
            if link.startswith("http"):
                story_dict['url'] = link
            else:
                story_dict['url'] = main_url + link
        except:
            story_dict['url'] = ''
            pass
        teasers_list.append(story_dict)

    time.sleep(1)

kathimerini_teasers_df = pd.DataFrame(teasers_list)

In [ ]:
kathimerini_teasers_df.to_csv('/content/gdrive/MyDrive/DATA_JOUR/KATHIMERINI_SCRAPING')

In [ ]:
full_articles_list = []

for article_url in kathimerini_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')
  article = doc.find('section', {'class': 'article__main'})
  full_article_dict = {}

  full_article_dict['site'] = "https://www.kathimerini.gr/"

  full_article_dict['url'] = article_url

  try:
    section = doc.find('span', {'class': 'nx-single-category-title'}).text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  try:
    title = doc.find('h1',{'class': 'entry-title'} ).text
    full_article_dict['title'] = title
  except:
    full_article_dict['title'] = ''
    pass

  try:
    date = doc.find('time', {'class': 'entry-date'})['datetime']
    full_article_dict['date'] = date
  except:
    full_article_dict['date'] = ''
    pass

  try:
    author = doc.find('span', {'class': 'author'}).text.replace('\n', '')
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  try:
    p_texts_list = []
    paragraphs = doc.find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [ ]:
kathimerini_full_articles_df = pd.DataFrame(full_articles_list)
kathimerini_full_articles_df

,site,url,section,title,date,author,full_text
0,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562512973/...,Πολιτική,Λοβέρδος: Πράξη απελευθέρωσης η αποχώρηση από ...,2023-07-10T10:01:36+03:00,Newsroom,«Πράξη απελευθέρωσης χαρακτήρισε» την απομάκρυ...
1,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562510909/...,Πολιτική,Το σχέδιο για μη κρατικά ΑΕΙ,2023-07-10T09:45:00+03:00,Απόστολος Λακασάς,Ποια βήματα θα ακολουθήσει η κυβέρνηση για να ...
2,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562512937/...,Πολιτική,Κούλογλου: Πρόταση για διαρχία στον ΣΥΡΙΖΑ – «...,2023-07-10T09:32:26+03:00,Newsroom,«Αυτό που χρειάζεται πάνω από όλα είναι επαγγε...
3,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562510969/...,Πολιτική,«Εμφραγμα» στη Δικαιοσύνη: Ενας εισαγγελέας γι...,2023-07-10T09:30:54+03:00,Ιωάννα Μάνδρου,Ερευνα - σοκ της Εισαγγελίας Πρωτοδικών Αθηνών...
4,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562512850/...,Πολιτική,ΣΥΡΙΖΑ: Συνεδριάζει η Πολιτική Γραμματεία – Στ...,2023-07-10T08:02:12+03:00,Newsroom,Η Πολιτική Γραμματεία αναμένεται να εξετάσει κ...
...,...,...,...,...,...,...,...
385,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562489576/...,ΠΡΟΕΔΡΙΚΟ ΜΕΓΑΡΟ,Δευτέρα στις 13:00 ορκίζεται ο Μητσοτάκης,2023-06-25T23:12:46+03:00,Newsroom,Ο πρόεδρος της Νέας Δημοκρατίας Κυριάκος Μητσο...
386,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562489567/...,Πολιτική,Εκλογές 2023: Χαμένοι και κερδισμένοι της ΝΔ σ...,2023-06-25T22:55:05+03:00,Newsroom,Ανακατατάξεις στους βουλευτές που εκλέγονται σ...
387,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562489561/...,Πολιτική,Πηγές ΣΥΡΙΖΑ: Ξεκινάνε άμεσα οι κομματικές διε...,2023-06-25T22:53:30+03:00,Newsroom,"Συγκεκριμένα, σύμφωνα με κομματικές πηγές, θα ..."
388,https://www.kathimerini.gr/,https://www.kathimerini.gr/politics/562489546/...,Πολιτική,Εκλογές 2023: Χαμένοι και κερδισμένοι του ΣΥΡΙ...,2023-06-25T22:45:46+03:00,Newsroom,Με μειωμένη δύναμη κατά 24 έδρες από τις εκλογ...


In [ ]:
kathimerini_full_articles_df.drop_duplicates(inplace=True)

In [ ]:
kathimerini_full_articles_df.to_csv('/content/gdrive/MyDrive/DATA_JOUR/FINAL ESSAY/KATHIMERINI_FULL.csv', index=False)